# http://neuralnetworksanddeeplearning.com/chap3.html

## An improved version of first notebook mnist_numpy, implementing the stochastic gradient descent learning algorithm for a feedforward neural network.  Improvements include the addition of the cross-entropy cost function, regularization, and better initialization of network weights.

In [61]:
import numpy as np
import random

In [62]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

In [63]:
class CrossEntropyCost(object):
    
    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.  Note that np.nan_to_num is used to ensure numerical
        stability.  In particular, if both ``a`` and ``y`` have a 1.0
        in the same slot, then the expression (1-y)*np.log(1-a)
        returns nan.  The np.nan_to_num ensures that that is converted
        to the correct value (0.0)."""
        return np.sum(np.nan_to_num(-y*np.log(a) - (a-y)*np.log(1-a)))
    
    @staticmethod
    def delta(a, y):
        """Return the error delta from the output layer."""
        return a-y

In [64]:
class Network:
    
    def __init__(self, sizes, cost=CrossEntropyCost):
        self.layer_count = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost = cost
        
    def default_weight_initializer(self):
        """http://neuralnetworksanddeeplearning.com/chap3.html#weight_initialization
        
        weights are initialized as Gaussian random variables with mean 0 and standard 
        deviation 1 divided by the square root of the number of connections input to 
        the neuron.
        
        biases are initliazed as Gaussian random varibales with mean 0 and standard
        deviation 1
        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x) for y, x in zip(self.sizes[1:], self.sizes[:-1])]
        
        
    def feedforward(self, a):
        for w, b in zip(self.weights, self.biases):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, train_data, epoch, mini_batch_size, lr, lmbda = 0.0):
        
        best_accuracy = 0.0
        epoch_to_wait = 0
        
        n = len(train_data)
        for i in xrange(epoch):
            random.shuffle(train_data)
            
            mini_batches = [train_data[k:k+mini_batch_size] for k in xrange(0, n, mini_batch_size)]
            
            for batch in mini_batches:
                self.update(batch, lr, lmbda, n)
                
            print("epoch {0} completed".format(i))
            
            accuracy = self.accuracy(train_data)/float(n)
            print "Accuracy on training data: {}".format(accuracy)
            
            if accuracy <= best_accuracy:
                epoch_to_wait += 1
                if epoch_to_wait == 10:
                    print("early stopping...")
                    break
            else:
                best_accuracy = accuracy
                epoch_to_wait = 0
            
    def update(self, batch, lr, lmbda, n, momentum=1.0):
        nebla_b = [np.zeros(b.shape) for b in self.biases]
        nebla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in batch:
            delta_b, delta_w = self.backprop(x, y)
            nebla_b = [nb + db for nb, db in zip(nebla_b, delta_b)]
            nebla_w = [nw + dw for nw, dw in zip(nebla_w, delta_w)]
            
        self.biases = [b - (lr/len(batch))*nb for b, nb in zip(self.biases, nebla_b)]
        # L2 regularization
        self.weights = [((1-lr*(lmbda/n)*momentum)*w) - (lr/len(batch))*nw for w, nw in zip(self.weights, nebla_w)]
    
    def backprop(self, x, y):
        nebla_w = [np.zeros(w.shape) for w in self.weights] # weight gradient for cost
        nebla_b = [np.zeros(b.shape) for b in self.biases] # bias gradient for cost
        
        # feedforward
        act = x
        acts = [x] # store all activations, layer-by-layer
        zs = [] # store all z vectors, layer-by-layer
                
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, act) + b
            zs.append(z)
            act = sigmoid(z)
            acts.append(act)
            
        # backward pass
        delta = self.cost.delta(acts[-1], y)
        
        nebla_b[-1] = delta
        nebla_w[-1] = np.dot(delta, acts[-2].T)
        
        for l in xrange(2, self.layer_count):
            z = zs[-l]
            z_derivative = sigmoid_derivative(z)
            delta = np.dot(self.weights[-l+1].T, delta) * z_derivative
            nebla_b[-l] = delta
            nebla_w[-l] = np.dot(delta, acts[-l-1].T)
        return (nebla_b, nebla_w)
        
    def accuracy(self, data):
        results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in data]
        return sum(int (x == y) for (x, y) in results)
    

In [65]:
import mnist_loader

train_data, val_data, test_data = mnist_loader.load_data_wrapper()

net = Network([784, 30, 10], cost=CrossEntropyCost)

net.SGD(train_data, 30, 10, 0.5, lmbda=5.0)

epoch 0 completed
Accuracy on training data: 0.94214
epoch 1 completed
Accuracy on training data: 0.94474
epoch 2 completed
Accuracy on training data: 0.95874
epoch 3 completed
Accuracy on training data: 0.9552
epoch 4 completed
Accuracy on training data: 0.96018
epoch 5 completed
Accuracy on training data: 0.96458
epoch 6 completed
Accuracy on training data: 0.96138
epoch 7 completed
Accuracy on training data: 0.96496
epoch 8 completed
Accuracy on training data: 0.96252
epoch 9 completed
Accuracy on training data: 0.96526
epoch 10 completed
Accuracy on training data: 0.9649
epoch 11 completed
Accuracy on training data: 0.94932
epoch 12 completed
Accuracy on training data: 0.96716
epoch 13 completed
Accuracy on training data: 0.96158
epoch 14 completed
Accuracy on training data: 0.9654
epoch 15 completed
Accuracy on training data: 0.9625
epoch 16 completed
Accuracy on training data: 0.9641
epoch 17 completed
Accuracy on training data: 0.96614
epoch 18 completed
Accuracy on training dat